In [2]:
import pandas as pd
from pyvi import ViTokenizer
import re
import nltk
import seaborn as sns
from matplotlib import pyplot

# 1. Import data

In [47]:
train_filename = "train_nor_811.xlsx"
valid_filename = "valid_nor_811.xlsx"
test_filename = "test_nor_811.xlsx"
train_data = pd.read_excel(train_filename, engine = "openpyxl")
valid_data = pd.read_excel(valid_filename, engine = "openpyxl")
test_data = pd.read_excel(test_filename, engine = "openpyxl")

In [48]:
from sklearn.preprocessing import LabelEncoder
def file_processing(data):
    data.drop(columns = {"Unnamed: 0"}, axis = 1, inplace = True)
    data["emotion_encode"] = data["Emotion"]
    encoder = LabelEncoder()
    data.emotion_encode = encoder.fit_transform(data.Emotion)
    return data

In [49]:
train_data = file_processing(train_data)
valid_data = file_processing(valid_data)
test_data = file_processing(test_data)

# 2. Data visualization

# 3. Data preprocessing

In [27]:
import json
f = open("sensitive_words.json", encoding="utf-8")
sensitive_words = json.load(f)
f.close()

In [28]:
def remove_duplicate(word):
    prev_char = ""
    clean_word = ""
    for character in word:
        if(character != prev_char):
            clean_word += character
            prev_char = character
    return clean_word

In [29]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def normalize_sentences(sentences):
    punc_lst = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''", "=", "%", "^", "@", "<", ">"}
    confusing_words = {"per", "dume"}
    acronym_word = {
        "ko" : "không",
        "k" : "không",
        "z" : "vậy",
        "dzậy" : "vậy",
        "dậy": "vậy",
        "t" : "tao",
        "m" : "mày",
        "sgk" : "sách_giáo_khoa",
        "zi" : "vậy",
        "dth" : "dễ_thương"
        
    }
    clean_sentences = []
    
    # remove punctuation and lowercase
    for sent in sentences:
        
        # remove emojis
        sent = deEmojify(sent)
        
        sent = nltk.word_tokenize(sent)
        temp = []
        for word in sent:
            word = word.lower()
            word = remove_duplicate(word)
            if (word in punc_lst or word in confusing_words):
                continue
            elif(word in acronym_word):
                temp.append(acronym_word[word])
            elif(word in sensitive_words):
                temp.append("<SEN>")
            elif(word.isdigit()):
                temp.append("<NUM>")
            else:
                temp.append(word)
        # remove whitespace
        sent = ' '.join(temp)
        
        clean_sentences.append(sent)
        
    return clean_sentences

In [25]:
def normalize_dataset(data):
    sentences = []
    for i in range(len(data)):
        sentences.append(ViTokenizer.tokenize(data.Sentence[i]))
    
    sentences = normalize_sentences(sentences)
    encode_tags = data.emotion_encode
    
    # remove empty sentences
    for idx, sent in enumerate(sentences):
        if sent.strip() == "":
            del sentences[idx]
            del encode_tags[idx]
    
    return sentences, encode_tags

In [50]:
train_clean_sentences, train_encode_tags = normalize_dataset(train_data)
valid_clean_sentences, valid_encode_tags = normalize_dataset(valid_data)
test_clean_sentences, test_encode_tags = normalize_dataset(test_data)

In [53]:
type(train_clean_sentences + valid_clean_sentences)

list

# 4. Model architecture

In [34]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers, activations , models , preprocessing , utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [54]:
MAX_LEN = 150

tokenizer = Tokenizer(lower = True, split = ' ')
tokenizer.fit_on_texts(train_clean_sentences + valid_clean_sentences)

X_train = tokenizer.texts_to_sequences(train_clean_sentences)
X_train = pad_sequences(X_train, MAX_LEN, padding='post', truncating='post')

X_valid = tokenizer.texts_to_sequences(valid_clean_sentences)
X_valid = pad_sequences(X_valid, MAX_LEN, padding='post', truncating='post')

print(X.shape)

(5547, 150)


In [55]:
vocab_size = len(tokenizer.word_index) + 1

## 4.1 CNN Model

In [61]:
embedding_dim = 128

inputs = layers.Input(shape=( MAX_LEN , ))
embedding = layers.Embedding(vocab_size, embedding_dim, input_length=MAX_LEN)(inputs)

cnn1 = layers.Conv1D(filters=100, kernel_size=1, activation='relu')(embedding)
cnn1 = layers.MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

cnn2 = layers.Conv1D(filters=100, kernel_size=2, activation='relu')(embedding)
cnn2 = layers.MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

outputs = layers.Concatenate()([cnn1,cnn2])

outputs = layers.Dense(28, activation='tanh')(outputs)
outputs = layers.Dense(14, activation='tanh')(outputs)
outputs = layers.Dense(7, activation='softmax')(outputs)
model=models.Model(inputs,outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# model.summary()

In [62]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, train_encode_tags,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_valid, valid_encode_tags),
                    batch_size=32)

Epoch 1/10
174/174 [==============================] - 10s 60ms/step - loss: 5.1332 - accuracy: 0.1147 - val_loss: 4.9339 - val_accuracy: 0.1312
Epoch 2/10
174/174 [==============================] - 10s 58ms/step - loss: 5.1309 - accuracy: 0.1192 - val_loss: 4.9339 - val_accuracy: 0.1283
Epoch 3/10
174/174 [==============================] - 10s 55ms/step - loss: 5.1309 - accuracy: 0.1213 - val_loss: 4.9339 - val_accuracy: 0.1443
Epoch 4/10
174/174 [==============================] - 10s 59ms/step - loss: 5.1308 - accuracy: 0.1318 - val_loss: 4.9339 - val_accuracy: 0.1224
Epoch 5/10
174/174 [==============================] - 9s 54ms/step - loss: 5.1308 - accuracy: 0.1271 - val_loss: 4.9339 - val_accuracy: 0.1370
Epoch 6/10
174/174 [==============================] - 10s 55ms/step - loss: 5.1308 - accuracy: 0.1312 - val_loss: 4.9339 - val_accuracy: 0.1327
Epoch 00006: early stopping


## 4.2 LSTM Model

In [67]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(7, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
y_train_lstm = tf.keras.utils.to_categorical(train_encode_tags)
y_valid_lstm = tf.keras.utils.to_categorical(valid_encode_tags)
history2 = model2.fit(X_train, y_train_lstm,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_valid, y_valid_lstm),
                    batch_size=512)

Epoch 1/10
11/11 [==============================] - 57s 5s/step - loss: 1.8365 - accuracy: 0.2690 - val_loss: 1.7726 - val_accuracy: 0.3120
Epoch 2/10
11/11 [==============================] - 64s 6s/step - loss: 1.7824 - accuracy: 0.2809 - val_loss: 1.7429 - val_accuracy: 0.3120
Epoch 3/10
11/11 [==============================] - 77s 7s/step - loss: 1.7788 - accuracy: 0.2809 - val_loss: 1.7485 - val_accuracy: 0.3120
Epoch 00003: early stopping


## 4. BiLSTM model

In [64]:
bi_model = Sequential()
bi_model.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
bi_model.add(SpatialDropout1D(0.2))
bi_model.add(layers.Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2)))
bi_model.add(Dense(7, activation='softmax'))
bi_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [66]:
bi_history = bi_model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10


ValueError: in user code:

    c:\python\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\python\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\python\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\python\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\python\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\python\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(da

In [49]:
bi_model.save("bi_model.h5")

## 4.3 Decision Tree Model + TF IDF vectorize

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [79]:
# TF IDF vectorize
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf = True,max_features=1837)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(train_clean_sentences)

tfidf_vectorizer_test = TfidfVectorizer(use_idf = True,max_features=1837)
tfidf_vectorizer_vectors_test = tfidf_vectorizer_test.fit_transform(test_clean_sentences)
print(tfidf_vectorizer_vectors_test.shape)

(693, 1837)


In [43]:
tree_model = DecisionTreeClassifier()
Y_DT = encode_tags

kf = KFold(n_splits=10)
for train_index, test_index in kf.split(tfidf_vectorizer_vectors):
    X_train_DT, X_test_DT = tfidf_vectorizer_vectors[train_index], tfidf_vectorizer_vectors[test_index]
    y_train_DT, y_test_DT = Y_DT[train_index], Y_DT[test_index]
    
    tree_model.fit(X_train_DT, y_train_DT)

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([4141], dtype='int64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [46]:
y_pred_DT = tree_model.predict(X_test_DT)

In [47]:
print(classification_report(y_test_DT, y_pred_DT))

              precision    recall  f1-score   support

           0       0.19      0.23      0.21        39
           1       0.40      0.42      0.41       101
           2       0.47      0.43      0.45       173
           3       0.39      0.27      0.32        33
           4       0.22      0.23      0.22       101
           5       0.32      0.39      0.35        82
           6       0.25      0.16      0.20        25

    accuracy                           0.35       554
   macro avg       0.32      0.30      0.31       554
weighted avg       0.36      0.35      0.35       554



## 4.4 Linear SVC + TF IDF vectorize

In [73]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [48]:
from  sklearn.model_selection import train_test_split
Y_DT = encode_tags
X_train_DT, X_test_DT, y_train_DT, y_test_DT = train_test_split(tfidf_vectorizer_vectors, Y_DT, test_size = 0.25, random_state = 36)

In [80]:
linear_svc = LinearSVC()
clf = linear_svc.fit(tfidf_vectorizer_vectors, train_encode_tags)

svc_model = CalibratedClassifierCV(base_estimator=linear_svc, cv="prefit")

svc_model.fit(tfidf_vectorizer_vectors, train_encode_tags)
y_pred_DT = svc_model.predict(tfidf_vectorizer_vectors_test)

In [81]:
print(classification_report(test_encode_tags, y_pred_DT))

              precision    recall  f1-score   support

           0       0.06      0.03      0.03        40
           1       0.22      0.27      0.24       132
           2       0.29      0.35      0.32       193
           3       0.00      0.00      0.00        46
           4       0.24      0.26      0.25       129
           5       0.19      0.17      0.18       116
           6       0.00      0.00      0.00        37

    accuracy                           0.23       693
   macro avg       0.14      0.15      0.15       693
weighted avg       0.20      0.23      0.21       693

